# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_city_data = pd.read_csv("output_data/cities.csv")
clean_city_data.head()

,Unnamed: 0,Date,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,0,1618704343,torbay,CA,47.6666,-52.7314,35.01,100,90,13.80
1,1,1618704310,chokurdakh,RU,70.6333,147.9167,17.11,86,100,5.21
2,2,1618704054,busselton,AU,-33.6500,115.3333,62.28,69,0,7.07
3,3,1618704658,isangel,VU,-19.5500,169.2667,82.40,78,75,8.05
4,4,1618704328,high level,CA,58.5169,-117.1360,44.60,49,75,10.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = clean_city_data[["Latitude","Longitude"]]
humidity = clean_city_data["Humidity (%)"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=5, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_weather = clean_city_data.loc[(clean_city_data["Max Temperature (F)"] < 80) & (clean_city_data["Max Temperature (F)"] > 55) \
                                    & (clean_city_data["Humidity (%)"] < 50) \
                                    & (clean_city_data["Wind Speed (mph)"] < 20) \
                                    & (clean_city_data["Cloudiness (%)"] < 40)].dropna()
city_weather

,Unnamed: 0,Date,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
22,22,1618704832,south lake tahoe,US,38.9332,-119.9844,60.80,19,1,13.80
27,27,1618704834,safut,JO,32.0336,35.8332,69.80,37,15,5.50
42,42,1618704339,hermanus,ZA,-34.4187,19.2345,69.01,34,0,7.09
60,60,1618704845,mahendragarh,IN,28.2833,76.1500,72.34,41,0,3.40
69,69,1618704848,yazman,PK,29.1212,71.7446,78.40,30,0,5.64
73,73,1618704408,saint george,US,37.1041,-113.5841,64.99,19,1,18.41
81,81,1618704657,alice springs,AU,-23.7000,133.8833,60.80,41,0,11.50
101,101,1618704860,spartanburg,US,34.9496,-81.9321,64.40,34,1,3.44
154,154,1618704345,aswan,EG,24.0934,32.9070,77.05,20,0,8.34
163,163,1618704880,chubbuck,US,42.9208,-112.4661,57.20,21,1,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = city_weather[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
22,south lake tahoe,US,38.9332,-119.9844,
27,safut,JO,32.0336,35.8332,
42,hermanus,ZA,-34.4187,19.2345,
60,mahendragarh,IN,28.2833,76.1500,
69,yazman,PK,29.1212,71.7446,
73,saint george,US,37.1041,-113.5841,
81,alice springs,AU,-23.7000,133.8833,
101,spartanburg,US,34.9496,-81.9321,
154,aswan,EG,24.0934,32.9070,
163,chubbuck,US,42.9208,-112.4661,


In [7]:
#Set parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    params["location"] = f"{latitude},{longitude}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name=requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field, moving on")
hotel_df

Missing field, moving on
Missing field, moving on


,City,Country,Latitude,Longitude,Hotel Name
22,south lake tahoe,US,38.9332,-119.9844,Tahoe Lakeshore Lodge & Spa
27,safut,JO,32.0336,35.8332,Amman International Hotel
42,hermanus,ZA,-34.4187,19.2345,Misty Waves Boutique Hotel
60,mahendragarh,IN,28.2833,76.1500,Yadav Dharamshala
69,yazman,PK,29.1212,71.7446,Jamal sons showroom
73,saint george,US,37.1041,-113.5841,Best Western Plus Abbey Inn
81,alice springs,AU,-23.7000,133.8833,Desert Palms Alice Springs
101,spartanburg,US,34.9496,-81.9321,Spartanburg Marriott
154,aswan,EG,24.0934,32.9070,Nuba Nile
163,chubbuck,US,42.9208,-112.4661,Clarion Inn Pocatello


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))